In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import re

In [3]:
raw_data=pd.read_csv("./IMDB Dataset.csv")
raw_data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
raw_data=raw_data.drop_duplicates()
raw_data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
raw_data.isna

<bound method DataFrame.isna of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[49582 rows x 2 columns]>

In [6]:
raw_data['sentiment']=pd.get_dummies(raw_data['sentiment'])['positive'].values

<ipython-input-6-5fffa0737a32>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data['sentiment']=pd.get_dummies(raw_data['sentiment'])['positive'].values


In [7]:
raw_data

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [8]:
stemmer=PorterStemmer()
vectorizer=TfidfVectorizer(max_features=5000,analyzer='word', encoding='utf-8')

In [9]:
stopwords=set(nltk.corpus.stopwords.words('english'))
c=1
cleaned_text=[]
c+=1
for i in raw_data['review'].values.tolist():
    i=BeautifulSoup(i,'lxml').text
    i=re.sub('[^a-zA-z]',' ',i).lower()
    words=[]
    for j in i.split():
        if j not in stopwords:
            words.append(stemmer.stem(j))
    cleaned_text.append(' '.join(words))

In [15]:
vectors=vectorizer.fit_transform(cleaned_text)

In [11]:
import pickle

In [12]:
with open('vectorizer.pickle','wb') as f:
    pickle.dump(vectorizer,f,protocol=2)

In [16]:
model=RandomForestClassifier()
model.fit(vectors,raw_data['sentiment'])


RandomForestClassifier()

In [19]:
def clean_text(review):
    review=BeautifulSoup(review,'lxml').text
    review=re.sub('[^a-zA-z]',' ',review).lower()
    words=[]
    for j in review.split():
        if j not in stopwords:
            words.append(stemmer.stem(j))
    return ' '.join(words)

def predict_class(review):
    review=vectorizer.transform([clean_text(review)])
    result=model.predict(review)[0]
    return result

In [27]:
predict_class('terriblly good')

1

In [28]:
with open('model.pickle','wb') as f:
    pickle.dump(model,f,protocol=2)